MNISTをpytorchで自力で実装するだけのプログラム 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

google colab 用に使用
URLのhttps://github.com を https://colab.research.google.com/github に変換する

In [2]:
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST

ModuleNotFoundError: No module named 'torchvision'

In [ ]:
mnist_data = MNIST('~/tmp/mnist', train=True, download=True, transform=transforms.ToTensor())
data_loader = DataLoader(mnist_data,batch_size=4,shuffle=False)
data_iterator = iter(data_loader)

images, labels = data_iterator.next()
data = images[2].numpy()
plt.imshow(data.reshape(28, 28), cmap='inferno', interpolation='bicubic')
plt.show()
print('ラベル:', labels[0])

とりあえず中身の可視化

In [0]:
# 学習データ
train_data_with_label = MNIST('~/tmp/mnist', train=True, download=True, transform=transforms.ToTensor())
train_data_loader = DataLoader(train_data_with_label,batch_size=1000,shuffle=True)
# 検証データ
test_data_with_label = MNIST('~/tmp/mnist', train=False, download=True, transform=transforms.ToTensor())
test_data_loader = DataLoader(test_data_with_label,batch_size=1000,shuffle=False)

In [0]:
import torch.nn as nn
class Flatten(nn.Module): # なんで無いの
    def forward(self, input):
        return input.view(input.size(0), -1)

In [0]:
from torch.autograd import Variable
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        # クラスの初期化
        super(CNN, self).__init__()
        # 画像を畳み込みを行うまで
        self.head = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=10, kernel_size=(5, 5), stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(p=0.2),
            nn.Conv2d(in_channels=10, out_channels=20, kernel_size=(5, 5), stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        # 畳み込みで得られたベクトルを出力層に順伝播させるまで
        self.tail = nn.Sequential(
            nn.Linear(320, 50),
            nn.ReLU(),
            nn.Linear(50, 50),
            nn.ReLU(),
            nn.Linear(50, 10)
        )
 
    def __call__(self, x):
        # 順伝播を計算する関数
        # :param x: 入力値
        h = self.head(x)
        h = h.view(-1, 320)
        h = self.tail(h)
        y = F.log_softmax(h)
        return y
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

model = CNN().cuda()
#model.to("cuda:0")
#model = net.Net(params).cuda() if params.cuda else net.Net(params)
#model.cuda()
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

In [0]:
from torch import optim
 
# 誤差関数の設定
loss_fn = nn.CrossEntropyLoss()  # 変数名にはcriterionも使われる
 
# 重みを学習する際の最適化手法の選択
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [0]:
# 5. 学習と推論の設定
# 5-1. 学習1回でやることを定義します
# Chainerのtraining.Trainer()に対応するものはない
 
from torch.autograd import Variable
 
 
def train(epoch):
    model.train()  # ネットワークを学習モードに切り替える
 
    # データローダーから1ミニバッチずつ取り出して計算する
    for data, target in train_data_loader:
        data, target = Variable(data).cuda() , Variable(target).cuda()   # 微分可能に変換
        #print(data.size())
        optimizer.zero_grad()  # 一度計算された勾配結果を0にリセット
        #data = data.view(1, -1)
        #print('02')
        output = model(data)  # 入力dataをinputし、出力を求める
        #print('02')
        loss = loss_fn(output, target)  # 出力と訓練データの正解との誤差を求める
        loss.backward()  # 誤差のバックプロパゲーションを求める
        optimizer.step()  # バックプロパゲーションの値で重みを更新する
    print('epoch %d, loss: %.4f'
              % (epoch, loss))
    #print("epoch{}：終了\n".format(epoch))

In [0]:
# 5. 学習と推論の設定
# 5-2. 推論1回でやることを定義します
# Chainerのtrainer.extend(extensions.Evaluator())に対応するものはない
 
 
def test():
    model.eval()  # ネットワークを推論モードに切り替える
    correct = 0
 
    # データローダーから1ミニバッチずつ取り出して計算する
    for data, target in test_data_loader:
        data, target = Variable(data).cuda() , Variable(target).cuda()   # 微分可能に変換
        output = model(data)  # 入力dataをinputし、出力を求める
 
        # 推論する
        pred = output.data.max(1, keepdim=True)[1]  # 出力ラベルを求める
        correct += pred.eq(target.data.view_as(pred)).sum()  # 正解と一緒だったらカウントアップ
 
    # 正解率を出力
    data_num = len(test_data_loader.dataset)  # データの総数
    print('\nテストデータの正解率: {}/{} ({:.0f}%)\n'.format(correct,
                                                   data_num, 100. * correct / data_num))

In [ ]:
# 6. 学習と推論の実行
for epoch in range(100):
    train(epoch)
 
test()